# Исследование надежности заемщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных**
 - children — количество детей в семье
 - days_employed — общий трудовой стаж в днях
 - dob_years — возраст клиента в годах
 - education — уровень образования клиента
 - education_id — идентификатор уровня образования
 - family_status — семейное положение
 - family_status_id — идентификатор семейного положения
 - gender — пол клиента
 - income_type — тип занятости
 - debt — имел ли задолженность по возврату кредитов
 - total_income — ежемесячный доход
 - purpose — цель получения кредита

### Шаг 1. Обзор данных

In [62]:
import pandas as pd
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer 
from collections import Counter

In [63]:
data = pd.read_csv('/datasets/data.csv')

In [64]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


 *Для начала **переименуем столбцы** для удобства их понимания:*
 
 - dob_years -> age
 - gender -> sex
        

In [65]:
data = data.rename(columns={'dob_years':'age', 'gender':'sex'})
data.head()


,children,days_employed,age,education,education_id,family_status,family_status_id,sex,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


*Получим **общую информацию**:*

In [66]:
data.describe()

,children,days_employed,age,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


<div style="border:solid purple 2px; padding: 20px">

**Наблюдения:**


1. Минимальное значение по 'children'-  -1. 
2. Максимальное значение по 'children' - 20. Относительно данных по 'children' похоже не человеческий фактор 
3. Отрицательные значения по количеству отработанных дней 'days_employed' 
4. Среднее значение по количеству отработанных дней 'days_employed' - 63046 дней (173 года, выглядит нереалистично)
5. Возраст по некоторым строкам 'age' равен 0 

In [67]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
age                 21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
sex                 21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<div style="border:solid purple 2px; padding: 20px">

**Наблюдения:**

1. Отсутствуют значения в столбцах 'days_employed' и 'total_income'.
2. Столбцам 'days_employed' и 'total_income' стоит присвоить целочисленные значения

**Проверим**, одни и те же строки отсутствуют по обоим столбцам ('days_employed' и 'total_income')?

In [68]:
data[(data['total_income'].isnull() == True) & (data['days_employed'].isnull() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
age                 2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
sex                 2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


**Предположение подтвердилось** - данные отсутствуют в строках 'days_employed' там же, где отсутствуют в строках 'total_income'

<div style="border:solid green 2px; padding: 20px">

**Вывод:**

- отрицательные данные 'days_employed' - возьмем весь столбец по модулю
- нулевой возраст для некоторых клиентов - предполагаю,что данные просто были не внесены в базу 
- "-1 ребенок" - исправить
- "20 детей" -  исправить

### Шаг 2.1 Заполнение пропусков

*Создадим pivot_table по каждому из типов занятости 'income_type' для того, чтобы понять девиацию со стажем (отрицательный стаж и стаж равный 1000 годам)*

Для каждого типа 'income_type' выведем следующее:
- общее кол-во строк данного типа в таблице;
- среднее значение по 'days_employed';
- для проверки корректности вычисления среднего и применения abs() выведем кол-во значений по 'days_employed' > 0 по данному 'income_type'.

In [69]:
data_pivot = data.groupby('income_type').agg({'days_employed':['count', 'mean', lambda x: sum(x>0)]})

dict_to_rename = dict(zip(data_pivot.columns.levels[1], ['количество строк', 'cреднее', 'кол-во значений > 0']))

data_pivot = data_pivot.rename(columns=dict_to_rename, level=1)
data_pivot

days_employed                                   
                количество строк        cреднее кол-во значений > 0
income_type                                                        
безработный                    2  366413.652744                 2.0
в декрете                      1   -3296.759962                 0.0
госслужащий                 1312   -3399.896902                 0.0
компаньон                   4577   -2111.524398                 0.0
пенсионер                   3443  365003.491245              3443.0
предприниматель                1    -520.848083                 0.0
сотрудник                  10014   -2326.499216                 0.0
студент                        1    -578.751554                 0.0

<div style="border:solid purple 2px; padding: 20px">

**Наблюдения:**
    
По "безработным" и "пенсионерам" значения > 0, по остальным - < 0.  Посчитаем среднее значение - применим abs().

Возьмём значения по 'days_employed' и 'income_type' по модулю.
Добавим столбец 'avr_days_employed' для вычисления коэффициента полноты количества отработанных дней (трудовой стаж официально может начинаться с 16 лет), далее заполним NaN по 'days_employed' средними показателями.
(избегаем столбцов, где в столбце 'days_employed' NaN через notnull())

In [70]:
data[['total_income', 'days_employed']] = data[['total_income', 'days_employed']].abs()
data['avr_days_employed'] = data[data['days_employed'].notnull()]['days_employed']/((data['age']-16)*365)
data.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,sex,income_type,debt,total_income,purpose,avr_days_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,0.889112
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0.551343
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,0.906273
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,0.706292
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,25.195563


Заполняем пропуски NaN в датафрейме data:
-'days_employed': NaN заменяем на произведение среднего коэффициента по группе 'income_type', умноженного на возраст в днях;
-'total_income': NaN заменяем на среднее значение по зарплате по группе 'income_type'.

In [71]:
data['days_employed'] = data.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.mean()*data['age']*365))
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))
data.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,sex,income_type,debt,total_income,purpose,avr_days_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,0.889112
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0.551343
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,0.906273
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,0.706292
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,25.195563


Вспомогательный столбец можем удалить.
Выведем общую информацию по датафрейму.

In [72]:
data = data.drop('avr_days_employed', axis=1)
data.info(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
age                 21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
sex                 21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.2 Проверка данных на аномалии и исправления.

*перейдем к аномальным значениям*

- преобразовать 'days_employed' по пенсионерам > 90 лет 
- необходимо привести значения в столбце 'education' к общему виду  (напр., 'Среднее'-'среднее'-'СРЕДНЕЕ') 
- исправить минимальное значение по 'children' (-1) на 1;
- исправить максимальное значение по 'children' (20) на 2;
- обработать нулевой возраст по некоторым строкам 'age'.

In [73]:
data['education'] = data['education'].str.lower()

In [74]:
print("Количество строк с 'children' -1 -", data[data['children'] == -1].count()[0])
print("Количество строк с 'children' 0 -", data[data['children'] == 0].count()[0])
print("Количество строк с 'children' 20 -", data[data['children'] == 20].count()[0])

Количество строк с 'children' -1 - 47
Количество строк с 'children' 0 - 14149
Количество строк с 'children' 20 - 76


In [75]:
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)

In [76]:
data.head(-10)

,children,days_employed,age,education,education_id,family_status,family_status_id,sex,income_type,debt,total_income,purpose
0,1,8.437673e+03,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4.024804e+03,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5.623423e+03,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4.124747e+03,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,3.402661e+05,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21510,2,2.377682e+07,28,среднее,1,женат / замужем,0,F,сотрудник,0,161380.260488,приобретение автомобиля
21511,0,6.125691e+02,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068.472941,покупка жилья для сдачи
21512,0,1.653778e+02,26,высшее,0,Не женат / не замужем,4,M,компаньон,0,147301.457769,получение дополнительного образования
21513,0,1.166217e+03,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986.142309,покупка жилья


*Проверим количество нулей по возрасту ('age') и количество несовершеннолетних*

In [77]:
print("Количество строк с 'age' 0 -", data[data['age'] == 0].count()[0])
print("Количество строк с 'age' < 19 -", data[data['age'] < 19].count()[0])

Количество строк с 'age' 0 - 101
Количество строк с 'age' < 19 - 101


Количества совпадают! Заполним значения по ним средним по каждой группе.

In [78]:
data['age'] = data.groupby('income_type')['age'].transform(lambda x: x.replace(0, int(x.mean())))
print("Количество строк с 'age' 0 -", data[data['age'] == 0].count()[0])

Количество строк с 'age' 0 - 0


In [79]:
data.describe()

,children,days_employed,age,education_id,family_status_id,debt,total_income
count,21525.000000,2.152500e+04,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.479721,1.522834e+08,43.495238,0.817236,0.972544,0.080883,1.673959e+05
std,0.755528,1.078074e+09,12.230322,0.548138,1.420324,0.272661,9.790695e+04
min,0.000000,0.000000e+00,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1.024652e+03,34.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,2.605748e+03,43.000000,1.000000,0.000000,0.000000,1.519313e+05
75%,1.000000,3.336411e+05,53.000000,1.000000,1.000000,0.000000,2.024175e+05
max,5.000000,9.725518e+09,75.000000,4.000000,4.000000,1.000000,2.265604e+06


<div style="border:solid green 2px; padding: 20px">

**Вывод:**
    
Все значения NaN по датафрейму заполнены, явно некорректные значения откорректированы

In [80]:
print('Столбец sex', data['sex'].unique())

Столбец sex ['F' 'M' 'XNA']


In [81]:
data[data['sex'] == 'XNA']

,children,days_employed,age,education,education_id,family_status,family_status_id,sex,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Судя по всему, клиент не передал информацию при выборе M и F. Это случайный характер ошибки.
Поскольку пол XNA только один, а в наших данных большинство женщины - заменим значением по умолчанию - F

In [82]:
data.loc[data['sex'] == 'XNA', 'sex'] = 'F'
# проверим
data.loc[data['sex'] == 'XNA']['sex'].count()

0

In [83]:
data.days_employed.describe().astype('int')

count         21525
mean      152283436
std      1078074117
min               0
25%            1024
50%            2605
75%          333641
max      9725518024
Name: days_employed, dtype: int64

In [84]:
data.age.max() 

75

In [85]:
(75-16) * 365

21535

In [86]:
def conv_days_employed(row):
    
    if row > 21535:
        row = row / 24 
        
    else: row
        
    return row
   
data['days_employed'] = data['days_employed'].apply(conv_days_employed)
print (data['days_employed'].head(10))

0     8437.673028
1     4024.803754
2     5623.422610
3     4124.747207
4    14177.753002
5      926.185831
6     2879.202052
7      152.779569
8     6929.865299
9     2188.756445
Name: days_employed, dtype: float64


### Шаг 2.3. Изменение типов данных.

*Изменим тип данных* 

Месячную зарплату('total_income') и стаж в днях ('days_employed') переводим в 'int'.


In [87]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

In [88]:
data.dtypes

children             int64
days_employed        int64
age                  int64
education           object
education_id         int64
family_status       object
family_status_id     int64
sex                 object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

### Шаг 2.4. Удаление дубликатов.

*посчитаем количество дубликатов*

In [89]:
data.duplicated().sum()

71

In [90]:
data[data.duplicated(keep=False)].sort_values(by=['total_income', 'days_employed'])

,children,days_employed,age,education,education_id,family_status,family_status_id,sex,income_type,debt,total_income,purpose
17787,0,299759117,54,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции с жильем
21415,0,299759117,54,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции с жильем
3344,0,310861306,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции со своей недвижимостью
9627,0,310861306,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции со своей недвижимостью
13300,0,310861306,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
19369,0,1445074,45,среднее,1,гражданский брак,1,F,компаньон,0,202417,свадьба
9920,0,1637751,51,среднее,1,гражданский брак,1,F,компаньон,0,202417,на проведение свадьбы
15991,0,1637751,51,среднее,1,гражданский брак,1,F,компаньон,0,202417,на проведение свадьбы
2254,0,1734089,54,высшее,0,женат / замужем,0,M,компаньон,0,202417,операции с коммерческой недвижимостью


<div style="border:solid purple 2px; padding: 20px">

**Наблюдения:**
    
Предполагаю, что данные ошибочно задвоились.
Удалим их.

In [91]:
data = data.drop_duplicates().reset_index(drop = True)
data.duplicated().sum()

0

### Шаг 2.5. Классификация по возрастным группам

*распределим заемщиков по возрастным категориям*

In [92]:
print(data['age'].value_counts())

39    647
35    616
40    613
41    605
34    601
38    597
42    596
33    581
31    559
36    554
44    545
29    544
30    537
48    536
37    536
50    513
43    512
32    509
49    508
28    503
45    496
27    493
52    484
56    483
47    477
54    476
46    472
59    463
53    459
57    456
58    454
51    446
55    443
26    408
60    374
25    357
61    354
62    348
63    269
24    264
64    260
23    252
65    193
22    183
66    182
67    167
21    111
68     99
69     85
70     65
71     56
20     51
72     33
19     14
73      8
74      6
75      1
Name: age, dtype: int64


*Напишем функцию*, где
- до 31 года у нас будет молодёжь, 
- с 31 до 60 - средний возраст, 
- больше 60 лет - пенсионеры



In [93]:
def age_group(age):
    
    if (age <= 31):
        return 'молодёжь'
    if 31 < age <= 60:
        return 'средний возраст'
    if age > 60:
        return 'пенсионеры'
    return 'группа не определена'
data['age_group'] = data['age'].apply(age_group)

print(data['age_group'].value_counts())

средний возраст    15052
молодёжь            4276
пенсионеры          2126
Name: age_group, dtype: int64


<div style="border:solid green 2px; padding: 20px">

**Вывод:**

Видим, что более половины заемщиков - люди среднего возраста

### Шаг 2.6. Категоризация дохода.

*На основании диапазонов, указанных ниже, создадим столбец total_income_category с категориями:*
- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

In [30]:
def income_group(row):
    
    income = row['total_income']

    if income <= 30000:
        return 'E'
    
    if income <= 50000 and income > 30001:
        return 'D'
    
    if income <= 200000 and income > 50001:
        return 'C'
    
    if income <= 1000000 and income > 200001:
        return 'B'
    
    if income > 1000001:
        return 'A'

data['total_income_category'] = data.apply(income_group, axis=1)
data.head(10)

,children,days_employed,age,education,education_id,family_status,family_status_id,sex,income_type,debt,total_income,purpose,age_group,total_income_category
0,1,351,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,средний возраст,B
1,1,167,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,средний возраст,C
2,0,234,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,средний возраст,C
3,3,171,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,средний возраст,B
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,средний возраст,C
5,0,38,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,молодёжь,B
6,0,119,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,средний возраст,B
7,0,6,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,средний возраст,C
8,2,288,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,средний возраст,C
9,0,91,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,средний возраст,C


*посчитаем количество заемщиков по доходам:*

In [31]:
print(data['total_income_category'].value_counts())

C    15514
B     5542
D      350
A       25
E       22
Name: total_income_category, dtype: int64


<div style="border:solid green 2px; padding: 20px">

**Вывод:**
    
Мы распределили заемщиков по их доходу и увидели, что практически не имеем в базе заемщиков с низким доходом,  2/3 от числа всех составляют заемщики со средним доходом.

### Шаг 2.7. Категоризация целей кредита

*Здесь нам необходимо привести цели кредита к общей форме и посчитать количество по каждой*

Проведём лемматизацию значений столбца с целями кредита
результат соединим методом join()
с помощью  контейнера Counter подсчитаем частоту упоминаний слов в тексте

In [32]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
Counter(m.lemmatize(' '.join(data['purpose'])))

Counter({'покупка': 5897,
         ' ': 55023,
         'жилье': 4460,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 853,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'получение': 1314,
         'высокий': 1374,
         'подержанный': 111,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607,
         '\n': 1})

1. Создадим функцию которая возвращает строку с ключевыми словами цели кредита
2. Отобразим уникальные значения (цели кредита)
3. Создадим новый столбец (purpose_cat) с категориями на основании результатов лемматизации

In [33]:
m = Mystem()

def lemma_pur(purpose):
    lemma = ' ' .join(m.lemmatize(purpose))
    return lemma

data['purpose_word'] = data['purpose'].apply(lemma_pur)

data['purpose_word'].unique()


def purpose_cat(list):
    if 'автомобиль' in list:
        return "автомобиль"
    if "образование" in list:
        return "образование"
    if "свадьба" in list:
        return "свадьба"
    if "недвижимость" in list or "строительство" in list or "жилье" in list:
        return "недвижимость"
data['purpose_cat'] = data['purpose_word'].apply(purpose_cat)
data


,children,days_employed,age,education,education_id,family_status,family_status_id,sex,income_type,debt,total_income,purpose,age_group,total_income_category,purpose_word,purpose_cat
0,1,351,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,средний возраст,B,покупка жилье \n,недвижимость
1,1,167,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,средний возраст,C,приобретение автомобиль \n,автомобиль
2,0,234,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,средний возраст,C,покупка жилье \n,недвижимость
3,3,171,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,средний возраст,B,дополнительный образование \n,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,средний возраст,C,сыграть свадьба \n,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,188,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,средний возраст,B,операция с жилье \n,недвижимость
21450,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,пенсионеры,C,сделка с автомобиль \n,автомобиль
21451,1,88,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,средний возраст,C,недвижимость \n,недвижимость
21452,3,129,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,средний возраст,B,на покупка свой автомобиль \n,автомобиль


In [34]:
print(data['purpose_cat'].value_counts())

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_cat, dtype: int64


<div style="border:solid green 2px; padding: 20px">

**Вывод:**
    
 Приведение целей кредита к общей форме позволило определиться с самыми популярными целями для получения кредита: самым популярным является кредит на недвижимость, затем идут приорбетение автомобиля и получения образования, последним по популярности идёт кредит на свадьбу

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [35]:
print(data['children'].value_counts())

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64


In [36]:
print(data.groupby('children')['debt'].mean())

children
0    0.075438
1    0.091658
2    0.094925
3    0.081818
4    0.097561
5    0.000000
Name: debt, dtype: float64


*создадим функцию, которая присвоит ID заёмщикам*
- многодетные (от 3)
- малодетные (1-2)
- бездетные


In [37]:
def children_id(children):
    if children < 1:
        return 'без детей'
    if children < 3:
        return 'малодетные'
    return 'многодетные'
data['children_id'] = data['children'].apply(children_id)
print(data.groupby('children_id')['debt'].mean())

children_id
без детей      0.075438
малодетные     0.092654
многодетные    0.081579
Name: debt, dtype: float64


<div style="border:solid green 2px; padding: 20px">

**Вывод:**

Бездетные заемщики реже допускают просрочки по кредитам. Стоит отметить, что количество детей не влияет на возратность кредита. 

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [38]:
print(data.groupby('family_status')['debt'].mean())

family_status
Не женат / не замужем    0.097509
в разводе                0.071130
вдовец / вдова           0.065693
гражданский брак         0.093471
женат / замужем          0.075452
Name: debt, dtype: float64


<div style="border:solid green 2px; padding: 20px">

**Вывод:**
    
Из расчетов выше видно, что неженатые\незамужние, а также лица состоящие в гражданском браке в процентном соотношении чаще допускают задолженности по кредитам, из этого можно сделать вывод, что наиболее надежные заемщики с семьями, либо побывавшие в семейных отношениях.    

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [39]:
print(data.groupby('total_income_category')['debt'].mean())

total_income_category
A    0.080000
B    0.069650
C    0.085729
D    0.060000
E    0.090909
Name: debt, dtype: float64



- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

<div style="border:solid green 2px; padding: 20px">

**Вывод:**

Заемщики с более низким уровнем дохода < 50000р. более дисциплинированы и менее склонны нарушать обязательства по выплатам кредита.


In [40]:
# КОД РЕВЬЮВЕРА

data['total_income_category'] = pd.qcut(data['total_income'], 4, labels=["низкий", "средний", "выше среднего", "высокий"])
print(data['total_income_category'].value_counts())

display(data.pivot_table(index = 'total_income_category', values = 'debt', aggfunc = ['count', 'sum', 'mean']))



выше среднего    5818
низкий           5364
средний          5363
высокий          4909
Name: total_income_category, dtype: int64


,count,sum,mean
,debt,debt,debt
total_income_category,,,
низкий,5364,427,0.079605
средний,5363,475,0.088570
выше среднего,5818,494,0.084909
высокий,4909,345,0.070279


##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [41]:
print(data.groupby('purpose_cat')['debt'].mean())

purpose_cat
автомобиль      0.093590
недвижимость    0.072334
образование     0.092200
свадьба         0.080034
Name: debt, dtype: float64


<div style="border:solid green 2px; padding: 20px">

**Вывод:**


По кредитам на автомобиль и образование чаще образовываются просрочки. Кредиты на недвижмость хоть и являются наиболее крупными, но и заемщики, как показывают расчеты, подходят к ним более ответственно.

## Общий вывод:

<div style="border:solid green 2px; padding: 20px">
    
Целью проекта было выявить - влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.
Проанализировав данные, полученные в ходе проекта, можно сказать, что количество детей влияет на погащение кредита в худшую сторону, однако заемщики с официально оформленными отношениями (или которые были в официальном браке в прошлом) но без детей являются самыми ответственными.
